In [43]:
from instagrapi import Client
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px

from PIL import Image
import requests
from io import BytesIO

import pytesseract
import malaya


/home/faiz/anaconda3/envs/work/lib/python3.10/site-packages/malaya/tokenizer.py:214: FutureWarning:

Possible nested set at position 3397

/home/faiz/anaconda3/envs/work/lib/python3.10/site-packages/malaya/tokenizer.py:214: FutureWarning:

Possible nested set at position 3927



In [ ]:
cl = Client()
cl.login(USERNAME, PASSWORD)

True

In [ ]:
hashtag_dump = cl.hashtag_medias_recent_v1(HASHTAG, 50)

In [4]:
tmp = []
for i in range(0,50):
    tmp.append(hashtag_dump[i].model_dump())

hashtag_posts = pd.DataFrame(tmp)

In [ ]:
hashtag_posts

### Plotting location on the map 

In [24]:
def media_type_func(x,y):
    if x == 1:
        return "Photo"
    elif x == 2 and y == "feed":
        return "Video"
    elif x == 2 and y == "igtv":
        return "IGTV"
    elif x == 2 and y == "clips":
        return "Reel"
    else:
        return "Album"

In [ ]:
geo_df = pd.DataFrame()

geo_df["User"] = hashtag_posts["user"].str.get("username")
geo_df["Taken at"] = hashtag_posts["taken_at"]
geo_df["Type"] = hashtag_posts["media_type"]
geo_df["Product type"] = hashtag_posts["product_type"]

geo_df["Type"] = geo_df.apply(lambda x: media_type_func(x["Type"], x["Product type"]), axis=1)

geo_df["Lat"] = hashtag_posts["location"].str.get("lat")
geo_df["Lan"] = hashtag_posts["location"].str.get("lng")

In [42]:
fig = px.scatter_map(geo_df, lat="Lat", lon="Lan", map_style="carto-darkmatter", hover_name="User", hover_data="Taken at", color="Type", zoom=11)
fig.show()

### Text preprocessing

In [44]:
# Unix/Linux
tesseract_path = r"/usr/bin/tesseract"

# Set tesseract path
pytesseract.pytesseract.tesseract_cmd = tesseract_path


#### Extract text from photo

In [64]:
img_texts = []

for img_url in hashtag_posts["thumbnail_url"]:
    if img_url:
        response = requests.get(img_url)
        img_text = pytesseract.image_to_string(Image.open(BytesIO(response.content)))
        print(f"Extracted text: {img_text}")
        img_texts.append(img_text)
    else:
        img_texts.append("")

hashtag_posts["thumbnail_text"] = img_texts

Extracted text: 
Extracted text: Ma ALOPIN

DONATION

For our UniKL Alumni Flood Victims

Monetary Donations

BS cimB

Universiti Teknikal Mara Sdn Bhd

8000 2759 72

Remarks :
ALUMNI CARE <Campus Name> - [Your Name]

Any inquiries, please contact PIC.

Kindly forward the proof of payment to
PIC via WhatsApp for confirmation.

+ Mrs. Azween:- 012-749 8174

www.unikledu.my

Item Donations
ALUMNI CARE Box of LOVE

1. Beras SKG 6. Bihun

2. Minyak Masak Botol (1kg) 7. Mi Segera
— 3. Tepung gandum 8. Biskut

4. Sardin 9. Susu pekat

5. Teh

Collection Centers :
UniKL HQ/MIIT, Kuala Lumpur
« Mrs. Azween (CACS level 3) :- 012-749 8174

UniKL RCMP, Ipoh, Perak
« Mr. Azrul (CL office) :- 011-3247 4700

UniKL MSI, Kulim, Kedah
« Mr. Azlan (CL office) :- 019-414 5048

UnIKL Page

Extracted text: Uni

M&\RA B) UNIVERSITI
ennai ALUMNI |

CORPORATION

ALUMNI CARE SQUAD

VOLUNTEERS

Let us contribute to help
our UniKL Alumni affected
by the recent floods.

JOIN US

@ 012 - 749 8174
@uniklalumni


Ex

In [77]:
def preprocess_text(text):
    # this uses NLTK
    # tokens = word_tokenize(text.lower())

    # this uses malaya toolkit
    tokenizer = malaya.tokenizer.Tokenizer()    
    tokens = tokenizer.tokenize(text, lowercase=True)

    # remove alphanumeric word(s)
    filtered_tokens = [word for word in tokens if word.isalnum()]
    
    return ' '.join(filtered_tokens)

In [ ]:
hashtag_posts

In [82]:
texts = []
for i, caption in enumerate(hashtag_posts["caption_text"]):
    if hashtag_posts.loc[i, "thumbnail_text"] == "":
        texts.append(preprocess_text(caption))
    else:
        text = caption + hashtag_posts.loc[i, "thumbnail_text"]
        texts.append(preprocess_text(text))

texts

['d day convocation yeayyy pre convo shoot at bukit bintang kuala lumpur theme lights at the city shoot by',
 'sister dah convo dahh dah betul betul bersedia nak berkhidmat untuk masyarakat dah',
 'alumni care donation we re reaching out to support our unikl alumni affected by the recent floods your help can make a huge difference monetary donations universiti teknikal mara sdn bhd cimb 8000275972 remarks alumni care campus name your name note please send proof of your donation transaction to the pic via whatsapp item donations please contact the pic of the specified collection centers thank you for your generosity and support together we can help our community in this time of need should you have any inquiries please reach out to mrs azween at alopin donation for our unikl alumni flood victims monetary donations bs cimb universiti teknikal mara sdn bhd 8000 2759 72 remarks alumni care campus name your name any inquiries please contact pic kindly forward the proof of payment to pic via

#### Detect language

In [83]:
import operator
fast_text = malaya.language_detection.fasttext()

langs = []
lang_probas = []

for text in texts:
    prob_dict = fast_text.predict_proba([text])
    print(prob_dict)

    lang = max(prob_dict[0].items(), key=operator.itemgetter(1))[0]
    prob = max(prob_dict[0].items(), key=operator.itemgetter(1))[1]
    
    langs.append(lang)
    lang_probas.append(prob)

hashtag_posts["Language"] = langs
hashtag_posts["Language Probability"] = lang_probas

[{'standard-english': 0.0, 'local-english': 0.0, 'manglish': 0.0, 'standard-indonesian': 0.0, 'socialmedia-indonesian': 0.0, 'standard-malay': 0.0, 'local-malay': 0.7632781, 'standard-mandarin': 0.0, 'local-mandarin': 0.0, 'other': 0.0}]
[{'standard-english': 0.0, 'local-english': 0.0, 'manglish': 0.0, 'standard-indonesian': 0.0, 'socialmedia-indonesian': 0.0, 'standard-malay': 0.0, 'local-malay': 0.9847328, 'standard-mandarin': 0.0, 'local-mandarin': 0.0, 'other': 0.0}]
[{'standard-english': 0.0, 'local-english': 0.0, 'manglish': 0.45959678, 'standard-indonesian': 0.0, 'socialmedia-indonesian': 0.0, 'standard-malay': 0.0, 'local-malay': 0.0, 'standard-mandarin': 0.0, 'local-mandarin': 0.0, 'other': 0.0}]
[{'standard-english': 0.0, 'local-english': 0.0, 'manglish': 0.0, 'standard-indonesian': 0.0, 'socialmedia-indonesian': 0.0, 'standard-malay': 0.80992615, 'local-malay': 0.0, 'standard-mandarin': 0.0, 'local-mandarin': 0.0, 'other': 0.0}]
[{'standard-english': 0.7093655, 'local-englis